In [1]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [6]:
!/usr/local/bin/python3 -m pip install tweepy pandas

  Using cached tweepy-4.15.0-py3-none-any.whl.metadata (4.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
Using cached tweepy-4.15.0-py3-none-any.whl (99 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 11.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 10.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [8]:
import tweepy
print("tweepy imported successfully")

tweepy imported successfully


In [12]:
mv ~/Downloads/bitcoin-tweets.zip /Users/gaoyue/w7/bitcoin_price_prediction_2023106041_YUE-GAO/data/

mv: rename /Users/gaoyue/Downloads/bitcoin-tweets.zip to /Users/gaoyue/w7/bitcoin_price_prediction_2023106041_YUE-GAO/data/bitcoin-tweets.zip: No such file or directory


In [1]:
import csv

input_file = "../data/bitcoin_tweets_kaggle.csv"
output_file = "../data/bitcoin_tweets_kaggle_cleaned.csv"

# 打开文件，逐行读取
with open(input_file, 'r', encoding='latin1') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    reader = csv.reader(infile, lineterminator='\n')
    writer = csv.writer(outfile, lineterminator='\n')
    
    # 写入表头
    header = next(reader)  # 读取第一行（表头）
    writer.writerow(["date", "text"])  # 只保留 date 和 text 列
    
    # 找到 date 和 text 列的索引
    date_idx = header.index("date")
    text_idx = header.index("text")
    
    # 逐行处理
    count = 0
    for row in reader:
        try:
            # 只提取 date 和 text 列
            writer.writerow([row[date_idx], row[text_idx]])
            count += 1
            if count >= 1000:  # 只取 1000 条
                break
        except (IndexError, UnicodeDecodeError):
            # 跳过无法解析的行
            continue

print(f"Processed {count} rows and saved to {output_file}")

Processed 1000 rows and saved to ../data/bitcoin_tweets_kaggle_cleaned.csv


In [2]:
import pandas as pd

# 加载清理后的文件
kaggle_tweets = pd.read_csv("../data/bitcoin_tweets_kaggle_cleaned.csv")
kaggle_tweets.rename(columns={"date": "timestamp"}, inplace=True)
kaggle_tweets["language"] = "en"  # 添加语言列
tweets = kaggle_tweets.head(1000)  # 确保只取 1000 条

# 保存到 CSV
tweets.to_csv("../data/tweets.csv", index=False)
print(f"Collected {len(tweets)} tweets from Kaggle dataset and saved to data/tweets.csv")

Collected 1000 tweets from Kaggle dataset and saved to data/tweets.csv


In [3]:
pip install praw

  Using cached praw-7.8.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached prawcore-2.4.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl.metadata (2.3 kB)
Using cached praw-7.8.1-py3-none-any.whl (189 kB)
Using cached prawcore-2.4.0-py3-none-any.whl (17 kB)
Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.6/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import praw
import pandas as pd
from datetime import datetime

# 初始化 Reddit API 客户端
reddit = praw.Reddit(
    client_id="1iggtu7LVAL8N2ff2zg3DA",
    client_secret="miXByiooBGMjOvWJnoFUyzGjMUYWrA",
    user_agent="BitcoinBot/0.1 by Bright-Complex1396",
    username="Bright-Complex1396",
    password="MyNewRedditPass123"  # 使用新密码
)

# 采集 r/Bitcoin 子版块的帖子
subreddit = reddit.subreddit("Bitcoin")
posts = []

# 采集最新的 100 条帖子
for submission in subreddit.new(limit=100):
    posts.append({
        "timestamp": datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
        "text": submission.title + " " + (submission.selftext if submission.selftext else ""),
        "language": "en"
    })

# 转换为 DataFrame
reddit_posts = pd.DataFrame(posts)

# 保存到 CSV
reddit_posts.to_csv("../data/reddit_posts.csv", index=False)
print(f"Collected {len(reddit_posts)} Reddit posts and saved to data/reddit_posts.csv")

Collected 100 Reddit posts and saved to data/reddit_posts.csv


In [3]:
ls /Users/gaoyue/w7/bitcoin_price_prediction_2023106041_YUE-GAO/data/reddit_posts.csv

/Users/gaoyue/w7/bitcoin_price_prediction_2023106041_YUE-GAO/data/reddit_posts.csv


In [6]:
!head -n 5 /Users/gaoyue/w7/bitcoin_price_prediction_2023106041_YUE-GAO/data/reddit_posts.csv

timestamp,text,language
2025-04-21 15:26:27,"Bitcoin to Gold Ratio The simplest way to get rich: sell gold, buy bitcoin.",en
2025-04-21 15:08:46,"Daily Discussion, April 21, 2025 Please utilize this sticky thread for all general **Bitcoin** discussions! If you see posts on the front page or /r/Bitcoin/new which are better suited for this daily discussion thread, please help out by directing the OP to this thread instead. Thank you!

If you don't get an answer to your question, you can try phrasing it differently or commenting again tomorrow.


In [7]:
import requests
import pandas as pd

# NewsAPI 密钥
api_key = "232870d36c4c4d0883a0e466e70a611b"  # 您的 NewsAPI 密钥

# 请求 NewsAPI 数据
url = "https://newsapi.org/v2/everything"
params = {
    "q": "bitcoin",  # 搜索关键词
    "language": "en",  # 语言
    "sortBy": "publishedAt",  # 按发布日期排序
    "apiKey": api_key
}

response = requests.get(url, params=params)
news_data = response.json()

# 提取文章
articles = []
for article in news_data["articles"][:100]:  # 取前 100 条
    articles.append({
        "timestamp": article["publishedAt"],
        "text": article["title"] + " " + (article["description"] if article["description"] else ""),
        "language": "en"
    })

# 转换为 DataFrame
news_articles = pd.DataFrame(articles)

# 保存到 CSV
news_articles.to_csv("../data/news_articles.csv", index=False)
print(f"Collected {len(news_articles)} news articles and saved to data/news_articles.csv")

Collected 100 news articles and saved to data/news_articles.csv


In [8]:
# 方法 1：使用 Python 查看前 5 行
import pandas as pd

news_data = pd.read_csv("../data/news_articles.csv")
news_data.head(5)

,timestamp,text,language
0,2025-04-20T06:35:57Z,"Boeing jet returns to US from China, a victim ...",en
1,2025-04-20T06:25:09Z,"Bitcoin: With 2,106 whale wallets and counting...",en
2,2025-04-20T06:16:50Z,Short Interest in Valkyrie Bitcoin Miners ETF ...,en
3,2025-04-20T06:02:56Z,Hut 8 Corp. (TSE:HUT) Receives Consensus Recom...,en
4,2025-04-20T05:00:34Z,Public-private collaboration is shaping Pakist...,en


In [9]:
!head -n 5 /Users/gaoyue/w7/bitcoin_price_prediction_2023106041_YUE-GAO/data/news_articles.csv

timestamp,text,language
2025-04-20T06:35:57Z,"Boeing jet returns to US from China, a victim of Trump’s tariff war By Reuters ",en
2025-04-20T06:25:09Z,"Bitcoin: With 2,106 whale wallets and counting, BTC’s big buyers are back! What now? BTC sees a shift in market sentiment, with vote up or down turning green for the first time in four months.
- Bitcoin whales are back in the market, with whale addresses hitting a 2025 high of 2106.
Over the past months, Bitcoin [BTC] has experienced extreme …",en
